In [3]:
## Make a pickle file with the df for the given:
# exp (e.g. 'FR1')
# subs
# selected_period (e.g. 'encoding')

# this will be used to run createEventsForDF.py in parallel by Slurm

In [1]:
import pandas as pd; pd.set_option('display.max_columns', 30); pd.set_option('display.max_rows', 100)
import numpy as np
from cmlreaders import CMLReader, get_data_index
from ptsa.data.filters import ButterworthFilter, ResampleFilter, MorletWaveletFilter
import xarray as xarray
import sys
import os
import matplotlib.pyplot as plt
%matplotlib inline
from pylab import *
from copy import copy
from scipy import stats
from scipy.stats import zscore
import pickle
plt.rcParams['pdf.fonttype'] = 42; plt.rcParams['ps.fonttype'] = 42 # fix fonts for Illustrator
sys.path.append('/home1/john/Downloads/ptsa_plot-master/')
sys.path.append('/home1/john/johnModules')
from brain_labels import HPC_labels, ENT_labels, PHC_labels, temporal_lobe_labels,\
                         MFG_labels, IFG_labels, nonHPC_MTL_labels, ENTPHC_labels, AMY_labels, ACC_OF_MFC_labels
%load_ext autoreload
%autoreload
from general import *
from SWRmodule import *  

df = get_data_index("r1") # all RAM subjects

In /home1/john/anaconda3/envs/workshopJ/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home1/john/anaconda3/envs/workshopJ/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home1/john/anaconda3/envs/workshopJ/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home1/john/anaconda3/envs/workshopJ/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /

In [2]:
### PARAMS ###

exp = 'catFR1' # 'FR1' 'catFR1' 'RepFR1'

selected_period = 'surrounding_recall' # 'surrounding_recall' 'encoding'

### END PARAMS ###

In [3]:
# 575 FR sessions. first 18 of don't load so skip those 
exp_df = df[df.experiment==exp]
if exp == 'FR1':
    exp_df = exp_df[
                    ((df.subject!='R1015J') | (df.session!=0)) & 
                    ((df.subject!='R1063C') | (df.session!=1)) & 
                    ((df.subject!='R1093J') | (~df.session.isin([1,2]))) &
                    ((df.subject!='R1100D') | (~df.session.isin([0,1,2]))) &
                    ((df.subject!='R1120E') | (df.session!=0)) &
                    ((df.subject!='R1122E') | (df.session!=2)) &
                    ((df.subject!='R1154D') | (df.session!=0)) &
                    ((df.subject!='R1186P') | (df.session!=0)) &
                    ((df.subject!='R1201P') | (~df.session.isin([0,1]))) &
                    ((df.subject!='R1216E') | (~df.session.isin([0,1,2]))) &
                    ((df.subject!='R1277J') | (df.session!=0)) &
                    ((df.subject!='R1413D') | (df.session!=0)) & 
                    ((df.subject!='R1123C') | (df.session!=2)) & # artifacts that bleed through channels (see SWR FR1 prob sessions ppt)
                    ((df.subject!='R1151E') | (~df.session.isin([1,2]))) & # more bleed-through artifacts (see same ppt)
                    ((df.subject!='R1275D') | (df.session!=3))  # 3rd session an actual repeat of 2nd session (Paul should have removed from database by now)
    #                 (df.subject!='R1065J') # sub with 9000 trials
                   ] 
elif exp == 'catFR1': 
    exp_df = exp_df[
                    ((df.subject!='R1044J') | (df.session!=0)) & # too few trials to do pg pairwise corr
                    ((df.subject!='R1491T') | (~df.session.isin([1,3,5]))) & # too few trials to do pg pairwise corr
                    ((df.subject!='R1486J') | (~df.session.isin([4,5,6,7]))) & # repeated data...will be removed at some point... @@
                    ((df.subject!='R1501J') | (~df.session.isin([0,1,2,3,4,5]))) & # these weren't catFR1 (and they don't load right anyway)
                    ((df.subject!='R1235E') | (df.session!=0)) & # split EEG filenames error...documented on Asana
                    ((df.subject!='R1310J') | (df.session!=1)) & # session 1 is just a repeat of session 0
                    ((df.subject!='R1239E') | (df.session!=0)) # some correlated noise (can see in catFR1 problem sessions ppt)
    ]
elif exp == 'RepFR1':
    exp_df = exp_df[
                    (df.subject!='R1564J') # clearly something wrong with these EEG when looking at ripple raster
                    ]
exp_df.shape
# exp_df = exp_df[257:] # for catFR1 this is R1385E-onwwards
# exp_df = exp_df[472:] # for FR1 this is R1385E-onwwards
# exp_df = exp_df[468:] # for catFR1 this is R1525J-onwards
# exp_df

/home1/john/anaconda3/envs/workshopJ/lib/python3.7/site-packages/ipykernel_launcher.py:30: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


(615, 17)

In [4]:
unique_combinations = exp_df[['subject', 'session']].drop_duplicates()
num_unique_combinations = unique_combinations.shape[0]
print(f'The unique number of subject-session combinations is {num_unique_combinations}')
print(f"from {len(np.unique(exp_df.subject))} patients")

The unique number of subject-session combinations is 615
from 269 patients


In [5]:
sum(df.experiment=='ICatFR1')
sum(df.experiment=='catFR1')

68

631

In [6]:
total_catFR_df = df[df.experiment.apply(lambda x: x == 'ICatFR1' or x == 'catFR1')]
len(total_catFR_df)
len(np.unique(total_catFR_df['subject']))

699

307

In [7]:
len(np.unique(exp_df[476:]['subject']))
len(exp_df[476:]) # 106 from 28 catFR1

len(np.unique(total_catFR_df['subject']))
len(total_catFR_df) # 57 from 31 ICatFR1
# 192 sessions from 68 patients
len(np.unique(exp_df[:476]['subject']))
len(exp_df[:476]) # 106 from 28 catFR1

38

139

307

699

231

476

In [9]:
# sub lists I've been using to explore FR1 

# subs = ['R1010J'] #'R1385E'] # ['R1065J'] #
# subs = ['R1002P','R1003P','R1006P','R1010J','R1112M','R1163T'] # initial 6 for hippocampus

# subs = subs+['R1001P','R1092J','R1151E','R1320D'] # additional subs with issues that I resolved with various RR and corr thresholds
# subs = ['R1112M','R1018P','R1020J','R1022J','R1023J','R1026D','R1027J'] # 7 with entorhinal
# subs = ['R1010J','R1112M'] # original 2 # R1108J beatiful catFR1 data; R1065J magical subject (like with FR1)
# subs = ['R1112M','R1163T'] # subjects with FR1 and catFR1. SRs of 1000 and 500 tho
# subs = ['R1151E'] # all channels X sessions get removed but 3 after 60/180 hz line removal
# these should all be worked out now. See SWR FR1 problem sessions PPT on Box for details of each
# subs = ['R1299T','R1332M','R1397D','R1349T','R1339D','R1337E','R1275D','R1151E','R1123C','R1120E','R1045E'] # final surrounding_recall problem subs after looking at huge raster!
# subs = ['R1308T','R1338T','R1358T'] # these guys had weird SRs and some loading problems when I went to whole_retrieval analysis...fixed with rounding
# subs = ['R1023J','R1101T','R1148P','R1368T','R1323T','R1334T'] # single session subs with memory allocation errors in ClusterRun
# subs = ['R1006P','R1010J','R1030J','R1032D','R1049J','R1051J','R1052E','R1054J','R1093J','R1098D','R1104D','R1108J','R1111M',
#         'R1115T','R1118N','R1124J','R1207J','R1230J','R1316T','R1329T','R1323T','R1337E','R1367D'] # subs that didn't load on 2020-07-04 encoding run
# subs = ['R1368T','R1461T','R1463E'] # subs with particularly low localization.pair matches in pairs...see Excel in loading info on Box for details
# subs = ['R1433E','R1355T','R1367D','R1368T'] # subs with "ca3" or "dg"
# subs = ['R1001P'] # subject with over 8000 FR1 trials X electrodes
# bad_subs = ['R1433E','R1051J'] # no electrode cats
# subs = ['R1379E','R1385E','R1387E','R1394E','R1402E'] # partial beep subs
# subs = ['R1379E','R1402E','R1396T','R1395M','R1415T','R1389J','R1404E']  # test subs for eeg offset correction
# subs = ['R1054J','R1345D','R1048E','R1328E','R1308T', # first 2 are sr ≥ 1000. 3rd is 500 Hz.
#         'R1137E','R1136N','R1094T','R1122E','R1385E']#, # nice example FR1 subs used in Fig. 2
#         'R1405E','R1486J','R1501J'] # adding in 3 catFR only patients that will go into Fig. 4
# subs = ['R1120E','R1349T','R1397D','R1332M','R1299T'] # FR1 patients with electrode search ranges limited per SWR problem sessions doc on Box
# subs = ['R1269E','R1328E','R1367D','R1397D','R1405E','R1405E','R1447M','R1469D'] # catFR1 patients with electrode search changes limited
# subs = ['R1065J','R1492J','R1525J'] # killer subs
# subs = ['R1030J','R1035M'] # MFG subs
# subs = ['R1207J'] #'R1035M'] #['R1108J'] # R1065J # 'R1035M' #R1086M
# subs = ['R1111M','R1108J','R1191J','R1229M','R1247P','R1264P','R1261P','R1016M','R1065J','R1191J',
#         'R1190P','R1254E','R1337E','R1118N','R1153T','R1156D'] # FR1 patients that are getting encoding memory errors 
# subs = ['R1051J','R1118N','R1154D','R1207J','R1308T','R1311T','R1329T','R1334T','R1336T','R1334T',
#         'R1342M','R1212P','R1346T','R1311T','R1323T','R1329T','R1342M','R1346T','R1367D','R1569T'] # catFR1 memory error patients

# subs = ['R1001P','R1002P','R1003P','R1006P','R1010J','R1018P','R1020J','R1022J', # mimicking test session
#         'R1023J','R1026D','R1027J','R1030J','R1031M','R1032D','R1033D','R1034D']
# subs = ['R1367D'] #['R1525J'] #['R1367D'] #R1065J'

# # FR1 subs 2024-08-05
# subs = ['R1010J','R1054J','R1048E','R1328E','R1308T',
#         'R1137E','R1136N','R1122E','R1112M','R1108J'] # removed R1094T who used Spanish
# catFR1 subs 2024-08-05
# subs = ['R1065J','R1108J','R1112M','R1122E','R1136N','R1137E',
#        'R1308T','R1328E','R1405E','R1486J','R1492J','R1501J','R1525J',
#        'R1485J','R1530J','R1536J','R1541T']

subs = ['R1626S','R1617S','R1536J','R1530J','R1518T','R1527J']

sub_df = exp_df[(exp_df.subject.isin(subs))  & (exp_df.experiment == exp)] # all sessions for subs
# sub_df = df[(df.subject.isin(subs))  & (df.experiment == exp) & (df.session==1)] # single session
# sub_df = df[(df.experiment == exp) & (df.session == 0)] # all FR subs 
# sub_df = sub_df[0:1]
len(sub_df)
sub_df

31

,Recognition,all_events,contacts,experiment,import_type,localization,math_events,montage,original_experiment,original_session,pairs,ps4_events,session,subject,subject_alias,system_version,task_events
2380,NaN,protocols/r1/subjects/R1518T/experiments/catFR...,protocols/r1/subjects/R1518T/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1518T/experiments/catFR...,0,NaN,NaN,protocols/r1/subjects/R1518T/localizations/0/m...,NaN,0,R1518T,R1518T,3.4,protocols/r1/subjects/R1518T/experiments/catFR...
2381,NaN,protocols/r1/subjects/R1518T/experiments/catFR...,protocols/r1/subjects/R1518T/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1518T/experiments/catFR...,0,NaN,NaN,protocols/r1/subjects/R1518T/localizations/0/m...,NaN,1,R1518T,R1518T,3.4,protocols/r1/subjects/R1518T/experiments/catFR...
2382,NaN,protocols/r1/subjects/R1518T/experiments/catFR...,protocols/r1/subjects/R1518T/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1518T/experiments/catFR...,0,NaN,NaN,protocols/r1/subjects/R1518T/localizations/0/m...,NaN,2,R1518T,R1518T,3.4,protocols/r1/subjects/R1518T/experiments/catFR...
2433,NaN,protocols/r1/subjects/R1527J/experiments/catFR...,protocols/r1/subjects/R1527J/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1527J/experiments/catFR...,0,NaN,NaN,protocols/r1/subjects/R1527J/localizations/0/m...,NaN,0,R1527J,R1527J,3.4,protocols/r1/subjects/R1527J/experiments/catFR...
2434,NaN,protocols/r1/subjects/R1527J/experiments/catFR...,protocols/r1/subjects/R1527J/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1527J/experiments/catFR...,0,NaN,NaN,protocols/r1/subjects/R1527J/localizations/0/m...,NaN,1,R1527J,R1527J,3.4,protocols/r1/subjects/R1527J/experiments/catFR...
2435,NaN,protocols/r1/subjects/R1527J/experiments/catFR...,protocols/r1/subjects/R1527J/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1527J/experiments/catFR...,0,NaN,NaN,protocols/r1/subjects/R1527J/localizations/0/m...,NaN,2,R1527J,R1527J,3.4,protocols/r1/subjects/R1527J/experiments/catFR...
2436,NaN,protocols/r1/subjects/R1527J/experiments/catFR...,protocols/r1/subjects/R1527J/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1527J/experiments/catFR...,0,NaN,NaN,protocols/r1/subjects/R1527J/localizations/0/m...,NaN,3,R1527J,R1527J,3.4,protocols/r1/subjects/R1527J/experiments/catFR...
2437,NaN,protocols/r1/subjects/R1527J/experiments/catFR...,protocols/r1/subjects/R1527J/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1527J/experiments/catFR...,0,NaN,NaN,protocols/r1/subjects/R1527J/localizations/0/m...,NaN,4,R1527J,R1527J,3.4,protocols/r1/subjects/R1527J/experiments/catFR...
2438,NaN,protocols/r1/subjects/R1527J/experiments/catFR...,protocols/r1/subjects/R1527J/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1527J/experiments/catFR...,0,NaN,NaN,protocols/r1/subjects/R1527J/localizations/0/m...,NaN,5,R1527J,R1527J,3.4,protocols/r1/subjects/R1527J/experiments/catFR...
2452,NaN,protocols/r1/subjects/R1530J/experiments/catFR...,protocols/r1/subjects/R1530J/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1530J/experiments/catFR...,0,NaN,NaN,protocols/r1/subjects/R1530J/localizations/0/m...,NaN,0,R1530J,R1530J,3.4,protocols/r1/subjects/R1530J/experiments/catFR...


# in this case just take the full df for these 10 subs

In [9]:
final_df = exp_df #sub_df # exp_df

len(final_df)
final_df.head()
final_df.tail()

615

,Recognition,all_events,contacts,experiment,import_type,localization,math_events,montage,original_experiment,original_session,pairs,ps4_events,session,subject,subject_alias,system_version,task_events
32,NaN,protocols/r1/subjects/R1004D/experiments/catFR...,protocols/r1/subjects/R1004D/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1004D/experiments/catFR...,0,NaN,0,protocols/r1/subjects/R1004D/localizations/0/m...,NaN,0,R1004D,R1004D,NaN,protocols/r1/subjects/R1004D/experiments/catFR...
53,NaN,protocols/r1/subjects/R1013E/experiments/catFR...,protocols/r1/subjects/R1013E/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1013E/experiments/catFR...,1,NaN,0,protocols/r1/subjects/R1013E/localizations/0/m...,NaN,0,R1013E,R1013E_1,NaN,protocols/r1/subjects/R1013E/experiments/catFR...
66,NaN,protocols/r1/subjects/R1015J/experiments/catFR...,protocols/r1/subjects/R1015J/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1015J/experiments/catFR...,0,NaN,0,protocols/r1/subjects/R1015J/localizations/0/m...,NaN,0,R1015J,R1015J,NaN,protocols/r1/subjects/R1015J/experiments/catFR...
72,NaN,protocols/r1/subjects/R1016M/experiments/catFR...,protocols/r1/subjects/R1016M/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1016M/experiments/catFR...,0,NaN,0,protocols/r1/subjects/R1016M/localizations/0/m...,NaN,0,R1016M,R1016M,NaN,protocols/r1/subjects/R1016M/experiments/catFR...
73,NaN,protocols/r1/subjects/R1016M/experiments/catFR...,protocols/r1/subjects/R1016M/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1016M/experiments/catFR...,0,NaN,1,protocols/r1/subjects/R1016M/localizations/0/m...,NaN,1,R1016M,R1016M,NaN,protocols/r1/subjects/R1016M/experiments/catFR...


,Recognition,all_events,contacts,experiment,import_type,localization,math_events,montage,original_experiment,original_session,pairs,ps4_events,session,subject,subject_alias,system_version,task_events
3089,NaN,protocols/r1/subjects/R1683A/experiments/catFR...,protocols/r1/subjects/R1683A/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1683A/experiments/catFR...,0,NaN,NaN,protocols/r1/subjects/R1683A/localizations/0/m...,NaN,1,R1683A,R1683A,3.4,protocols/r1/subjects/R1683A/experiments/catFR...
3090,NaN,protocols/r1/subjects/R1683A/experiments/catFR...,protocols/r1/subjects/R1683A/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1683A/experiments/catFR...,0,NaN,NaN,protocols/r1/subjects/R1683A/localizations/0/m...,NaN,2,R1683A,R1683A,3.4,protocols/r1/subjects/R1683A/experiments/catFR...
3091,NaN,protocols/r1/subjects/R1683A/experiments/catFR...,protocols/r1/subjects/R1683A/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1683A/experiments/catFR...,0,NaN,NaN,protocols/r1/subjects/R1683A/localizations/0/m...,NaN,3,R1683A,R1683A,3.4,protocols/r1/subjects/R1683A/experiments/catFR...
3112,NaN,protocols/r1/subjects/R1687J/experiments/catFR...,NaN,catFR1,build,0,protocols/r1/subjects/R1687J/experiments/catFR...,0,NaN,NaN,NaN,NaN,0,R1687J,R1687J,4.0,protocols/r1/subjects/R1687J/experiments/catFR...
3122,NaN,protocols/r1/subjects/R1689J/experiments/catFR...,protocols/r1/subjects/R1689J/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1689J/experiments/catFR...,0,NaN,NaN,protocols/r1/subjects/R1689J/localizations/0/m...,NaN,0,R1689J,R1689J,4.0,protocols/r1/subjects/R1689J/experiments/catFR...


In [8]:
save_df = 0

if save_df == 1:

    import dill
    print("SAVING TEMP DF")
    with open(f'/scratch/john/SWRrefactored/patient_info/temp_dfSWR_{selected_period}_{exp}.p', 'wb') as f: 
        dill.dump(list(final_df.itertuples()) ,f)

    params = []
    for i in range(len(list(final_df.itertuples()) )):
        params.append(i)

    print("Number of sessions in final_df: ", len(params))
    print("From these subjects:")
    print(np.unique(final_df.subject))

SAVING TEMP DF


NameError: name 'final_df' is not defined

# to check out df for a specific region:

In [14]:
selected_region = ACC_OF_MFC_labels #ENTPHC_labels #ENTPHC_labels #

import warnings
# Ignore specific warnings
warnings.filterwarnings("ignore", category=UserWarning, message="Lab-specific readers may be moved to the cmlreaders package")
warnings.filterwarnings("ignore", category=UserWarning, message="Could not load MNI coordinates")
warnings.filterwarnings("ignore", category=UserWarning, message="Lab-specific readers may be moved to the cmlreaders package")
warnings.filterwarnings("ignore", category=FutureWarning, message="Could not load MNI coordinates")
warnings.filterwarnings("ignore", category=FutureWarning, message="Lab-specific readers may be moved to the cmlreaders package")
warnings.filterwarnings("ignore", category=FutureWarning, message="pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead")

remove_soz_ictal = 0

# get region label
if selected_region == HPC_labels:
    region_name = 'HPC'
elif selected_region == ENT_labels:
    region_name = 'ENT'
elif selected_region == PHC_labels:
    region_name = 'PHC'
elif selected_region == temporal_lobe_labels:
    region_name = 'TEMPORALLOBE'
elif selected_region == MFG_labels:
    region_name = 'MFG'
elif selected_region == IFG_labels:
    region_name = 'IFG'
elif selected_region == nonHPC_MTL_labels:
    region_name = 'nonHPC_MTL'    
elif selected_region == ENTPHC_labels:
    region_name = 'ENTPHC' 
elif selected_region == AMY_labels:
    region_name = 'AMY'

data = {
    'HPC_names': [],
    'sub_names': [],
    'sub_sess_names': [],
    'electrode_labels': [],
    'channel_nums': []
}

# Initialize a set to keep track of subjects that pass the if statement
passed_subjects = set()

for row in exp_df.itertuples(): #exp_df.itertuples(): #sub_df.itertuples():   
    sub = row.subject
    session = row.session
    exp = row.experiment
    mont = int(row.montage)
    loc = int(row.localization)
    reader = CMLReadDFRow(row)
    evs = reader.load('task_events')
    word_evs = evs[evs['type'] == 'WORD']

    pairs = reader.load('pairs')
    try:
        localizations = reader.load('localization')
    except:
        localizations = []
    tal_struct, bipolar_pairs, mpchans = get_bp_tal_struct(sub, montage=mont, localization=loc)
    elec_regions, atlas_type, pair_number, has_stein_das = get_elec_regions(localizations, pairs)

    try:
        elec_cats = reader.load('electrode_categories')
    except:
        if remove_soz_ictal == True:
            print("Remove soz ictal is true")
            add_session_to_exclude_list("Remove soz ictal is true")
            sys.exit()
        else: 
            elec_cats = []
  
    bad_bp_mask = getBadChannels(tal_struct, elec_cats, remove_soz_ictal)

    electrode_search_range = getElectrodeRanges(elec_regions, exp, sub, session, mont)

    for channel in electrode_search_range:
        if (elec_regions[channel] in selected_region) & (bad_bp_mask[channel] == 0):
            data['HPC_names'].append(elec_regions[channel])
            data['sub_names'].append(sub)
            data['sub_sess_names'].append(f"{sub}-{session}")
            data['electrode_labels'].append(pairs.iloc[channel].label)
            data['channel_nums'].append(channel)
#             print('Cumulative array size after channel: ' + str(channel))
            passed_subjects.add(sub)

cliff_notes_df = pd.DataFrame(data)

# # Create final_df
# selected_sub_df = sub_df[sub_df['subject'].isin(passed_subjects)]
# print('Done making df for region: '+region_name)
# selected_sub_df

In [15]:
print(f'Sub-sessions for {selected_region}')
np.unique(cliff_notes_df.sub_sess_names)
print('Unique subs:')
print(np.unique(cliff_notes_df.sub_names))

Sub-sessions for [' left aorg anterior orbital gyrus', 'left aorg anterior orbital gyrus', ' left lorg lateral orbital gyrus', 'left lorg lateral orbital gyrus', ' left porg posterior orbital gyrus', 'left porg posterior orbital gyrus', ' left morg medial orbital gyrus', 'left morg medial orbital gyrus', ' left gre gyrus rectus', 'left gre gyrus rectus', ' left mfc medial frontal cortex', 'left mfc medial frontal cortex', ' left acgg anterior cingulate gyrus', 'left acgg anterior cingulate gyrus', ' right aorg anterior orbital gyrus', 'right aorg anterior orbital gyrus', ' right lorg lateral orbital gyrus', 'right lorg lateral orbital gyrus', ' right porg posterior orbital gyrus', 'right porg posterior orbital gyrus', ' right morg medial orbital gyrus', 'right morg medial orbital gyrus', ' right gre gyrus rectus', 'right gre gyrus rectus', ' right mfc medial frontal cortex', 'right mfc medial frontal cortex', ' right acgg anterior cingulate gyrus', 'right acgg anterior cingulate gyrus'

array(['R1041M-0', 'R1045E-0', 'R1045E-1', 'R1061T-0', 'R1074M-0',
       'R1094T-0', 'R1105E-0', 'R1106M-0', 'R1131M-0', 'R1138T-0',
       'R1138T-1', 'R1157C-0', 'R1157C-1', 'R1157C-2', 'R1158T-0',
       'R1163T-0', 'R1169P-0', 'R1171M-0', 'R1174T-0', 'R1178P-0',
       'R1180C-0', 'R1180C-1', 'R1180C-2', 'R1192C-0', 'R1192C-1',
       'R1192C-2', 'R1204T-0', 'R1217T-0', 'R1227T-0', 'R1228M-0',
       'R1231M-0', 'R1239E-1', 'R1240T-0', 'R1243T-0', 'R1243T-1',
       'R1243T-2', 'R1243T-3', 'R1254E-0', 'R1264P-0', 'R1264P-1',
       'R1275D-0', 'R1279P-0', 'R1279P-1', 'R1288P-0', 'R1303E-0',
       'R1315T-0', 'R1320D-0', 'R1320D-1', 'R1320D-2', 'R1324M-0',
       'R1330D-0', 'R1330D-1', 'R1337E-0', 'R1337E-1', 'R1342M-0',
       'R1343J-1', 'R1347D-0', 'R1347D-1', 'R1347D-2', 'R1348J-1',
       'R1354E-0', 'R1354E-1', 'R1354E-2', 'R1354E-3', 'R1361C-0',
       'R1361C-1', 'R1361C-2', 'R1364C-0', 'R1364C-1', 'R1366J-0',
       'R1367D-0', 'R1368T-0', 'R1370E-0', 'R1372C-0', 'R1372C

Unique subs:
['R1041M' 'R1045E' 'R1061T' 'R1074M' 'R1094T' 'R1105E' 'R1106M' 'R1131M'
 'R1138T' 'R1157C' 'R1158T' 'R1163T' 'R1169P' 'R1171M' 'R1174T' 'R1178P'
 'R1180C' 'R1192C' 'R1204T' 'R1217T' 'R1227T' 'R1228M' 'R1231M' 'R1239E'
 'R1240T' 'R1243T' 'R1254E' 'R1264P' 'R1275D' 'R1279P' 'R1288P' 'R1303E'
 'R1315T' 'R1320D' 'R1324M' 'R1330D' 'R1337E' 'R1342M' 'R1343J' 'R1347D'
 'R1348J' 'R1354E' 'R1361C' 'R1364C' 'R1366J' 'R1367D' 'R1368T' 'R1370E'
 'R1372C' 'R1376D' 'R1378T' 'R1379E' 'R1380D' 'R1381T' 'R1382T' 'R1383J'
 'R1384J' 'R1385E' 'R1391T' 'R1394E' 'R1395M' 'R1396T' 'R1398J' 'R1400N'
 'R1401J' 'R1403N' 'R1404E' 'R1405E' 'R1406M' 'R1408N' 'R1409D' 'R1412M'
 'R1413D' 'R1414E' 'R1415T' 'R1420T' 'R1421M' 'R1422T' 'R1423E' 'R1426N'
 'R1427T' 'R1436J' 'R1441T' 'R1443D' 'R1444D' 'R1447M' 'R1449T' 'R1450D'
 'R1454M' 'R1456D' 'R1457T' 'R1459M' 'R1463E' 'R1465D' 'R1468J' 'R1469D'
 'R1472T' 'R1473J' 'R1476J' 'R1477J' 'R1478T' 'R1484T' 'R1485J' 'R1487T'
 'R1489E' 'R1490T' 'R1491T' 'R1493T' '

# test load to see if CMLReaders works with updates

In [10]:
# encoding params
encoding_whole_time = 1000*np.round(12*(1.6+0.875),1) # 0.875 is average of 0.75-1.0 s so 29.7 is average encoding length
encoding_time = 2300 # actual preentation is 1.6 s + 0.75-1.0 s so keep +700 ms so can plot +500 ms
pre_encoding_time = -700 # since minimum ISI is 0.75 s let's only plot the 500 ms before word on with a 200 ms buffer
# these aren't likely to be changed:
desired_sample_rate = 500. # in Hz. This seems like lowerst common denominator recording freq.
eeg_buffer = 300 # buffer to add to either end of IRI when processing eeg #**
align_adjust = 0

psth_start = pre_encoding_time
psth_end = encoding_time

for row in sub_df.itertuples(): #sub_df.itertuples():   
    try:
        sub = row.subject; session = row.session; exp = row.experiment
        mont = int(row.montage); loc = int(row.localization)
        reader = CMLReadDFRow(row)
        evs = reader.load('task_events')   

        pairs = reader.load('pairs')
        try:
            localizations = reader.load('localization')
        except:
            localizations = []
        tal_struct, bipolar_pairs, mpchans = get_bp_tal_struct(sub, montage=mont, localization=loc)
        elec_regions,atlas_type,pair_number,has_stein_das = get_elec_regions(localizations,pairs) 


        evs_encoding_words = evs[evs.type=='WORD']
        evs_encoding_words = evs_encoding_words[evs_encoding_words.list>-1]            
        evs_encoding_words = evs_encoding_words[evs_encoding_words.eegoffset>-1]

        # since finding all encoding words IN the list of correctly free recalled words won't have any intrusions
        eeg_events = evs_encoding_words



        eeg = reader.load_eeg(events=eeg_events, rel_start=psth_start-eeg_buffer+align_adjust, 
                              rel_stop=psth_end+eeg_buffer+align_adjust, clean=True, scheme=pairs) #**
        print(f'{sub}_{session}_loads with shape:')
        np.shape(eeg)
    except Exception as e:
        print(f'{sub}_{session}_does not load with error:')
        print(e)

/home1/john/anaconda3/envs/workshopJ/lib/python3.7/site-packages/cmlreaders/readers/electrodes.py:279: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  json_normalize(flat_contact_data).set_index('name'),
/home1/john/anaconda3/envs/workshopJ/lib/python3.7/site-packages/cmlreaders/readers/electrodes.py:280: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  json_normalize(flat_pairs_data).set_index('names')
/home1/john/anaconda3/envs/workshopJ/lib/python3.7/site-packages/ptsa/data/readers/__init__.py:19: FutureWarning: PTSA readers will be removed in a future release. Please consider using the cmlreaders package instead: https://github.com/pennmem/cmlreaders
  FutureWarning
/home1/john/anaconda3/envs/workshopJ/lib/python3.7/site-packages/ptsa/data/readers/tal.py:43: FutureWarning: Lab-specific readers may be moved to the cmlreaders package (https://github.com/pennmem/cmlreaders)
  FutureWarning)


R1518T_0_loads with shape:


(144, 145, 3600)

R1518T_1_loads with shape:


(144, 145, 3600)

...

R1617S_6_loads with shape:


(144, 186, 3600)

R1617S_7_loads with shape:


(144, 186, 3600)

R1626S_0_does not load with error:
len(channels) (189) must be the same length as the second data dimension (156)
R1626S_2_does not load with error:
len(channels) (189) must be the same length as the second data dimension (156)
R1626S_3_does not load with error:
len(channels) (189) must be the same length as the second data dimension (156)
R1626S_4_does not load with error:
len(channels) (189) must be the same length as the second data dimension (156)
R1626S_6_does not load with error:
len(channels) (189) must be the same length as the second data dimension (156)
R1626S_7_does not load with error:
len(channels) (189) must be the same length as the second data dimension (156)
R1626S_8_does not load with error:
len(channels) (189) must be the same length as the second data dimension (156)


In [11]:
len(sub_df)

31